In [1]:
import time
import requests
from selenium import webdriver
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import csv
import pickle
import numpy as np
import pandas as pd
import pickle
import jdatetime
#starting date = 29 tir

In [3]:
df = pd.read_csv("data.csv")

C:\Users\Arian_Gh\AppData\Local\Temp\ipykernel_13552\3477112522.py:1: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("data.csv")


In [5]:
df["company_size"].unique()

array(['11 تا 50 نفر', '51 تا 200 نفر', 'بیش از 5000 نفر',
       '501 تا 1000 نفر', '201 تا 500 نفر', '1001 تا 5000 نفر',
       'زیر 10 نفر', '11 - 50 employees', '501 - 1000 employees', nan,
       '1001 - 5000 employees', '51 - 200 employees',
       '201 - 500 employees', 'More than 5001 employees',
       'Less than 10 employees'], dtype=object)

In [6]:
df.columns

Index(['company_size', 'education', 'gender', 'salary', 'workdays', 'language',
       'business_trip', 'age', 'industry', 'bonus', 'location', 'ad_name',
       'softwares', 'military_services', 'experience', 'date'],
      dtype='object')

In [7]:
# A dictionary to map Persian/English categories to their midpoints
size_mapping = {
    '11 تا 50 نفر': '30', 
    '51 تا 200 نفر': '125', 
    'بیش از 5000 نفر': '5000+', 
    '501 تا 1000 نفر': '750', 
    '201 تا 500 نفر': '350', 
    '1001 تا 5000 نفر': '3000', 
    'زیر 10 نفر': '<10', 
    '11 - 50 employees': '30', 
    '501 - 1000 employees': '750', 
    '1001 - 5000 employees': '3000', 
    '51 - 200 employees': '125', 
    '201 - 500 employees': '350', 
    'More than 5001 employees': '5000+', 
    'Less than 10 employees': '<10'
}

# Replace the values using the map
df['company_size'] = df['company_size'].map(size_mapping)

# For NaN values, you could either fill them with a placeholder like 'Unknown'
df['company_size'].fillna('Unknown', inplace=True)

In [10]:
import numpy as np

def process_age(age):
    if pd.isnull(age):
        return [np.nan, np.nan]
    
    if ' - ' in age:
        min_age, max_age = age.split(' - ')
        return [int(min_age), int(max_age.split()[0])]
    
    if age.startswith('از '):
        min_age = age.split()[1]
        return [int(min_age), np.nan]

    if age.startswith('تا '):
        max_age = age.split()[1]
        return [np.nan, int(max_age)]
    
    if age.endswith('Years Old'):
        if 'Until' in age:
            max_age = age.split()[1]
            return [np.nan, int(max_age)]
        elif 'From' in age:
            min_age = age.split()[1]
            return [int(min_age), np.nan]
        else:
            min_age, max_age = age.split('-')
            return [int(min_age.strip()), int(max_age.split()[0].strip())]

df['min_age'], df['max_age'] = zip(*df['age'].map(process_age))

In [15]:
del df["age"]

In [17]:
df.head()

,company_size,education,gender,salary,workdays,language,business_trip,industry,bonus,location,ad_name,softwares,military_services,experience,date,min_age,max_age
0,30,['کارشناسی| مالی و حسابداری'],ترجیحاً خانم,NaN,شنبه تا چهارشنبه 8 الی 16,['انگلیسی| متوسط ۵۰٪'],-,نفت، گاز و پتروشیمی,-,تهران، فاطمی,کارشناس حسابداری,"['Microsoft Word| متوسط', 'Microsoft Excel| مت...",NaN,NaN,1402-03-23,23.0,33.0
1,125,NaN,فقط آقا,12 - 10 میلیون تومان,شنبه تا چهارشنبه 8 تا 17:33، پنجشنبه 8:30 تا ...,NaN,-,تولیدی / صنعتی,پاداش -بیمه درمان تکمیلی,رباط کریم,کارگر ساده - آقا,NaN,اتمام خدمت سربازی و یا معافیت از آن الزامی است,NaN,1402-02-26,23.0,45.0
2,5000+,NaN,تفاوتی ندارد,NaN,شنبه تا چهارشنبه 7 الی 16,NaN,-,بانکداری,وام -پاداش -بیمه درمان تکمیلی -تسهیلات تفری...,تهران، آرژانتین,کارشناس موشن گرافیست,"['Adobe Photoshop| پیشرفته', 'Adobe Illustrato...",اتمام خدمت سربازی و یا معافیت از آن الزامی است,NaN,1402-02-11,NaN,35.0
3,30,NaN,تفاوتی ندارد,20 - 15 میلیون تومان,10 تا 20,['انگلیسی| متوسط ۵۰٪'],-,سرمایه گذاری و مالی,پورسانت,تهران، جردن,مدیر اجرایی,"['Microsoft Word| متوسط', 'Microsoft Powerpoin...",NaN,NaN,1402-02-12,25.0,43.0
4,750,NaN,فقط آقا,NaN,شیفت شب ( 3 روز در هفته ),['انگلیسی| متوسط ۵۰٪'],-,سرمایه گذاری و مالی,بیمه درمان تکمیلی -اتاق بازی -ناهار -میان و...,کرج، حیدرآباد,کارشناس مرکز تماس ( شیفت شب) - آقا,NaN,اتمام خدمت سربازی و یا معافیت از آن الزامی است,NaN,1402-04-06,20.0,30.0


In [18]:
df.to_csv("Job.csv",index=False)